In [1]:
%matplotlib inline

import time
import numpy as np
import progressbar_renamed as progressbar
import copy
import networkx as nx
from networkx.algorithms import bipartite
from operator import itemgetter


import import_ipynb
import QAOA_for_QLS as qaoa
import Side_Calc_QLS as sc

importing Jupyter notebook from QAOA_for_QLS.ipynb
importing Jupyter notebook from Side_Calc_QLS.ipynb


In [2]:
def top_gains_populate_subset(lis, subset_size, gains):
    if len(lis) <= subset_size:
        return sorted(lis)
    return [
        x[0] for x in sorted(gains.items(), key=itemgetter(1))[-subset_size:]
    ]

In [9]:
def iteration_step(M, curr_solution, subset, lis):

    subset2lis = dict((x,lis.index(subset[x])) for x in range(0, len(subset)))
    node2lis=dict((lis[x],x) for x in range(len(lis)))
    
    n=len(subset)
    C = [0.0] * n
    for i in subset:
        for j in set(lis) - set(subset):
            C[subset.index(i)] += M[node2lis[i], node2lis[j]] * curr_solution[node2lis[j]]
                                                           
    indices = []  # rows and columns of M to keep for subset
    for i in range(n):
        indices.append(subset2lis[i])
    indices=np.array(indices)
    
    result = qaoa.qaoa_basic(len(subset), M[np.ix_(indices, indices)],C)
    optimized_subset=result.x
    
        
    for i in range(0, len(optimized_subset)):
        curr_solution[subset2lis[i]] = optimized_subset[i]
    return curr_solution

   

def QLS(M,lis,size_of_iteration=12,stopping_criteria=3):
    
    n=len(lis)
    # random initial guess
    curr_solution = list(np.random.randint(2, size=n))
    
    curr_energy = sc.compute_energy(M, curr_solution)
    
    all_time_best_solution = curr_solution
    all_time_best_energy = curr_energy

    visited = set()
    it = 0
    it_stuck = 0
    all_energies = []
    
    while (n - len(visited)):
        it += 1
        if it_stuck > stopping_criteria: 
            break
        gains_list = []
        for w in progressbar.progressbar(lis):
            gains_list.append(sc.compute_gain(M, curr_solution, lis.index(w), True))
        #print(gains_list)
        gains = {lis[v]: gain for gain, v in gains_list} 
       #print(gains)
        
        
        subset = top_gains_populate_subset(lis, size_of_iteration, gains)
        
        cand_solution = iteration_step(M, copy.deepcopy(curr_solution), list(subset),lis)
        cand_energy = sc.compute_energy(M, cand_solution)
        
        #print('it', it, 'cand_energy', cand_energy, 'curr_best',
              #all_time_best_energy)
        #return cand_energy, cand_solution
        
        all_energies.append({
            'it': it,
            'cand_energy': cand_energy,
            'curr_best': all_time_best_energy
        })
        if cand_energy > curr_energy:
            curr_energy = cand_energy
            curr_energy = cand_energy
            it_stuck = 0
        else:
            it_stuck += 1
        if curr_energy > all_time_best_energy:
            all_time_best_solution = curr_solution
            all_time_best_energy = curr_energy
        
        #break Bedingung?, vgl.: single_level_refinement
    return (all_time_best_energy, all_time_best_solution, it,
            all_energies)
        

In [11]:

M=[[1,1,1,0,0,0],[1,1,1,0,0,0],[1,1,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]
M=np.array(M)
#cur_solution=np.random.randint(2, size=12)
#cur_solution=[1,1,1,1,1,1,1,1,1,1,1,1]
#cur_solution=[0,1,1,0,1,1,0,1,1,0,0,0,]
#vorher=curr_solution
#print(cur_solution)
#subset=(18,37,68,25,52)
lis=(2,5,8,12,16,18)

QLS(M, lis)

100% (6 of 6) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


(12.0, [1, 1.0, 1.0, 1.0, 1, 1.0])